In [ ]:

!pip install torch==1.8.0
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
!pip install torch-geometric
!pip install torch-geometric-temporal
!pip install dgl

     |████████████████████████████████| 735.5 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.0 which is incompatible.
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu102.html


### Начало

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#папка с библиотеками (для колаба)
base = 'drive/My Drive/GPB'

In [ ]:
import sys
import re

import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
!pip install -Uqq ipdb
import ipdb # %pdb on для включения дебаг режима
from matplotlib import style
style.use('ggplot')
%config InlineBackend.figure_format = 'retina'

in_colab = 'google.colab' in sys.modules

if in_colab:
    sys.path.append(base)

def get_drive_path(path):
    in_colab = 'google.colab' in sys.modules
    return '/content/'+base+'/'+re.sub(r'\\','/',path) if in_colab else path

Matplotlib is building the font cache; this may take a moment.


In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-geometric
!pip install torch-geometric-temporal
!pip install dgl-cu102

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html


In [ ]:
import os
import pickle

import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import dgl

import gnn_package as gnn

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


### Elliptic bitcoin (Static)

In [ ]:
# Скачать файлы https://www.kaggle.com/ellipticco/elliptic-data-set
# в папку data/graph/elliptic_bitcoin_dataset

features_path = get_drive_path('data\\graph\\elliptic_bitcoin_dataset\\elliptic_txs_features.csv')
edges_path = get_drive_path('data\\graph\\elliptic_bitcoin_dataset\\elliptic_txs_edgelist.csv')
classes_path = get_drive_path('data\\graph\\elliptic_bitcoin_dataset\\elliptic_txs_classes.csv')

df_features, df_edges = gnn.data.load_and_preprocess_elliptic_dataset(features_path, edges_path, classes_path)

dgl_data = gnn.data.create_elliptic_dataset_graph(df_features, df_edges, create_masks=True, graph_type='dgl')

num_features = dgl_data.get_node_features().shape[1]
num_classes = 2

#### GCN

In [ ]:
num_hidden_features = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
model = gnn.static.GCN_DGL(num_features, num_hidden_features, num_classes)
model.train_(dgl_data, epochs=1, device=device, node_mask_attr='train_mask')
pred, f1 = model.evaluate(dgl_data, 'test_mask')
f1

  0%|          | 0/1 [00:00<?, ?it/s]

array([0.96204252, 0.15550756])

#### GCN + Neighbour Sampling

In [ ]:
import importlib
importlib.reload(gnn)

num_hidden_features = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataloader = gnn.data.create_nb_dataloader_dgl(dgl_data, dgl_data.train_mask, device)
test_dataloader = gnn.data.create_nb_dataloader_dgl(dgl_data, dgl_data.test_mask, device)

#device = 'cpu'
model = gnn.static.GCN_NB_DGL(num_features, num_hidden_features, num_classes)
model.train_(train_dataloader, epochs=1, device=device, node_mask_attr=None)
pred, f1 = model.evaluate(test_dataloader, 'test_mask')
f1

array([0.96410355, 0.        ])

### Elliptic bitcoin (Dynamic)

In [ ]:
graph_list_dgl = gnn.data.create_temporal_elliptic_dataset_graph(df_features, df_edges, 'dgl')
train_dataset_dgl, test_dataset_dgl = gnn.data.temporal_list_split(graph_list_dgl, train_ratio=0.75)

num_nodes = df_features.shape[0]
num_features = train_dataset_dgl[0].get_node_features().shape[1]
num_classes = len(np.unique(train_dataset_dgl[0].cpu().get_node_labels()))

# Примерные метрики моделей из статьи EvolveGCN
df_all_metrics_original = pd.DataFrame({'GCN':[0.55, 0.55, 0.5, 0.4, 0.45, 0.65, 0, 0.05, 0, 0, 0.07, 0, 0.07],
                                        'EvolveGCN-H':[0.55, 0.63, 0.5, 0.25, 0.1, 0.5, 0, 0, 0, 0, 0, 0.1, 0.25],
                                        'EvolveGCN-O':[0.6, 0.7, 0.7, 0.55, 0.7, 0.73, 0, 0.05, 0, 0, 0, 0, 0.05],
                                        'GCN-GRU':[0.63, 0.8, 0.7, 0.67, 0.65, 0.79, 0, 0.05, 0, 0, 0.07, 0, 0.06]})

df_all_metrics = pd.DataFrame()

#### GConvGRU

In [ ]:
import importlib
importlib.reload(gnn)
num_hidden_features = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'

model = gnn.dynamic.GConvGRU_DGL(num_features, num_hidden_features, num_classes)
model.to(device)
model.train_(train_dataset_dgl,
             epochs=1,
             device=device,
             node_mask_attr='node_mask')
preds, metrics = model.evaluate(test_dataset_dgl, 'node_mask')
print('\n Test:')
pd.DataFrame(metrics, columns=['f1 majority class', 'f1 minority class'])

  0%|          | 0/1 [00:00<?, ?it/s]


 Test:


,f1 majority class,f1 minority class
0,0.882353,0.310345
1,0.876555,0.460251
2,0.846784,0.264045
3,0.788540,0.251208
4,0.815900,0.125828
5,0.779275,0.275510
6,0.758940,0.056995
7,0.705666,0.015408
8,0.836609,0.013559
9,0.783740,0.000000


#### GConvGRU + Neighbour Sampling

In [ ]:
num_hidden_features = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'

train_batch_loader_list = [gnn.data.create_nb_dataloader_dgl(data, [4, 4], data.node_mask, device)
                           for data in train_dataset_dgl]
test_batch_loader_list = [gnn.data.create_nb_dataloader_dgl(data, [4, 4], data.node_mask, device)
                          for data in test_dataset_dgl]

model = gnn.dynamic.GConvGRU_NB_DGL(num_features, num_hidden_features, num_classes)
model.to(device)
model.train_(train_batch_loader_list,
             epochs=1,
             device=device,
             node_mask_attr='node_mask')
preds, metrics = model.evaluate(test_batch_loader_list, 'node_mask')
print('\n Test:')
pd.DataFrame(metrics, columns=['f1 majority class', 'f1 minority class'])



 Test:


,f1 majority class,f1 minority class
0,0.948207,0.093023
1,0.925043,0.200000
2,0.925654,0.089744
3,0.895611,0.092784
4,0.943902,0.041667
5,0.892540,0.090226
6,0.932356,0.016393
7,0.927757,0.000000
8,0.960922,0.025000
9,0.935989,0.000000


In [ ]:
pip install memory_profiler

In [ ]:
import os 
os.chdir('/content/drive/MyDrive/LiteralE-master')
!python -m memory_profiler temp1.py

Using backend: pytorch

[0.96410355 0.        ]
Filename: temp1.py

Line #    Mem usage    Increment  Occurences   Line Contents
    45    671.9 MiB    671.9 MiB           1   @profile
    46                                         def main():
    47    671.9 MiB      0.0 MiB           1     device='cpu'
    48    671.9 MiB      0.0 MiB           1     in_colab = 'google.colab' in sys.modules
    49                                         
    50    671.9 MiB      0.0 MiB           1     if in_colab:
    51                                             sys.path.append(base)
    52    671.9 MiB      0.0 MiB           1     features_path = get_drive_path('/content/drive/MyDrive/GPB/data/graph/elliptic_bitcoin_dataset/elliptic_txs_features.csv')
    53    671.9 MiB      0.0 MiB           1     edges_path = get_drive_path('/content/drive/MyDrive/GPB/data/graph/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv')
    54    671.9 MiB      0.0 MiB           1     classes_path = get_drive_path('

In [ ]:
df_features = pd.read_csv('/content/drive/MyDrive/GPB/data/graph/elliptic_bitcoin_dataset/elliptic_txs_features.csv', header=None)

In [ ]:
df_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,-0.167933,-0.049707,-0.164402,-0.028741,-0.035391,-0.042955,-0.013282,-0.057195,-0.169609,-0.171154,-0.174473,-1.373657,-1.371460,-0.139731,-0.148912,-0.080147,-0.155661,-0.010763,-0.012107,-0.139733,-0.148907,-0.080147,-0.155661,-0.010669,-0.012005,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.097719,-0.127462,0.003143,0.002426,-0.120950,-0.199145,-0.187993,-0.212948,1.064205,1.063787,-1.373782,-1.354735,-0.297975,-1.403698,1.342003,1.340733,-0.171601,-0.458162,-0.423588,-0.440883,-1.015963,-1.016230,-0.968903,-0.375715,0.759748,-0.768329,1.488113,1.487932,-0.216814,-0.605631,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,-0.167948,-0.049707,-0.164417,-0.028741,-0.035391,-0.042955,-0.013282,-0.055327,-0.169757,-0.171477,-0.174490,0.887058,0.884557,-0.139731,-0.148912,-0.080147,-0.155661,-0.010763,-0.012107,-0.139733,-0.148907,-0.080147,-0.155661,-0.010669,-0.012005,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.097719,-0.127462,0.003143,0.002426,-0.121330,-0.110933,-0.075909,-0.111641,-1.159649,-1.160129,-1.373723,-1.353918,-0.295982,-1.403215,-0.975738,-0.975237,-0.168742,-0.263290,-0.186389,-0.250875,-1.015963,-1.016230,-0.968903,0.146997,1.366287,-0.464773,-1.116918,-1.116948,-0.216814,0.634272,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,-0.168576,-0.049707,-0.165054,-0.028741,-0.035391,-0.042955,-0.013282,-0.055298,-0.170400,-0.172217,-0.175227,0.887058,0.884557,-0.139729,-0.148911,-0.080147,-0.155660,-0.010763,-0.012107,-0.139731,-0.148906,-0.080147,-0.155660,-0.010669,-0.012005,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.097719,-0.129496,0.003143,0.002426,-0.122974,-0.041556,0.012549,-0.032244,-1.159649,-1.160129,-1.373902,-1.356210,-0.301548,-1.404577,-0.975738,-0.975237,-0.168742,-0.192468,-0.099790,-0.182133,-1.015963,-1.016230,-0.968903,-1.421138,-0.453330,-1.375441,-1.116918,-1.116948,-0.216814,0.407161,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,-0.115831,0.043598,-0.163905,1.059993,2.397457,-0.063200,-0.013282,4.800240,0.174329,-0.172908,0.619399,-0.243300,-0.243452,-0.139732,-0.146895,-0.079667,-0.155554,0.773847,0.378516,-0.139734,-0.146891,-0.079667,-0.155553,0.773932,0.378593,-0.024669,-0.031271,-0.023044,-0.026215,0.101008,...,0.019771,-0.119195,0.147881,0.956170,0.759756,-0.041556,-0.188716,0.149949,-0.047722,-0.048171,-1.373723,-1.356131,-0.301754,-1.404445,0.183133,0.182748,-0.163577,-0.456561,-0.424672,-0.436915,0.143001,0.142525,-0.425896,-1.421138,-1.059868,-1.071885,0.185597,0.185492,-0.216814,-0.617907,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,0.041399,0.935886,-0.098695,1.203038,0.988902,0.341693,-0.013282,-0.057392,0.058329,-0.032930,-0.042709,0.065455,0.324947,-0.139732,-0.148854,-0.080105,-0.155638,1.313051,1.106965,-0.139734,-0.148850,-0.080104,-0.155638,1.313130,1.106995,-0.024669,-0.031272,-0.023045,-0.026215,0.001428,...,-0.077909,-0.093120,1.587690,1.586539,-0.101820,0.704558,0.448898,0.659662,0.311091,0.302198,-1.373962,-1.355393,-0.300941,-1.404583,0.064846,-0.035232,-0.168742,-0.442156,-0.416540,-0.418273,0.367605,0.392035,-0.9689